In [40]:
import numpy as np
import os
import itertools

from pathlib2 import Path
from bs4 import BeautifulSoup as Soup

contiki_dir = Path('..')
experiment_dir = Path(contiki_dir, 'tpwsn-trickle/experiments')
abs_dir = Path("/Users/david/Projects/contiki-ng/tpwsn-trickle/experiments")

script_template = Path(contiki_dir, 'tpwsn-trickle/sim-script.template.js')
sim_template = Path(contiki_dir, 'tpwsn-trickle/7x7.csc')

# Experiment params
fail_range = range(1,17)
fail_mode = ["random", "location"]

redundancy_range = [2] #range(2,4)
imin_range = [16] #[8, 16, 32]
imax_range = [10] #range(8, 11)
recovery_range = range(1, 16)

repeats = 16

experiment_size = 7 # Number of motes along one axis (forms a square)
experiment_space = list(itertools.product(fail_range, fail_mode, redundancy_range, imin_range, imax_range, recovery_range))

# open(str(script_template), 'r').readlines()
# open(str(sim_template), 'r').readlines()

In [107]:
def create_sim_script(params, size, seed, run):
    motes, mode, k, imin, imax, recovery = params
    mote_range = range(0, size**2)
    soup = Soup(open(str(sim_template), 'r').read())
    script = open(str(script_template), 'r').readlines()
    
    # Set the random seed of the simulation
    soup.findAll('randomseed')[0].string.replace_with(str(seed))
    
#     print(soup.findAll('script')[0].string)
    # TODO: Substitute the simulation script
    
    # Fix the plugin config for the radio timeline
    conf = soup.findAll('plugin_config')[2]
    motes_conf = conf.findChildren('mote')
    for mote in motes_conf:
        mote.decompose()
        
    for mote in mote_range:
        mote_conf_str = "<mote>{id}</mote>".format(id=(mote+1))
        conf.append(Soup(mote_conf_str))
    
    # Delete all current motes
    sim = soup.findAll('simulation')[0]
    motes_old = sim.findChildren('mote')
    for mote in motes_old:
        mote.decompose()
        
    # Add the motes back to the sim
    for mote in mote_range:
        mote_x = 40.0 * (mote % 7)
        mote_y = 40.0 * (mote // 7)
        mote_z = 0.0
        
        mote_str = """<mote>
  <breakpoints />
  <interface_config>
    org.contikios.cooja.interfaces.Position
    <x>{x}</x>
    <y>{y}</y>
    <z>{z}</z>
  </interface_config>
  <interface_config>
    org.contikios.cooja.mspmote.interfaces.MspClock
    <deviation>1.0</deviation>
  </interface_config>
  <interface_config>
    org.contikios.cooja.mspmote.interfaces.MspMoteID
    <id>{id}</id>
  </interface_config>
  <motetype_identifier>sky1</motetype_identifier>
</mote>""".format(x=mote_x, y=mote_y, z=mote_z, id=mote+1)
        sim.append(Soup(mote_str))
    
create_sim_script(experiment_space[0], experiment_size, 123456, 0)  

/**
 * TPWSN Trickle Simulation Script
 *
 * Created by David Richardson, University of Warwick
 */
load("nashorn:mozilla_compat.js");
importPackage(java.io);
importPackage(java.util);
importPackage(org.contikios.cooja.util);
importPackage(org.contikios.cooja.interfaces);

// Java types
var ArrayList = Java.type("java.util.ArrayList");
var HashSet = Java.type("java.util.HashSet");
var ArrayDeque = Java.type("java.util.ArrayDeque");

// The maximum number of nodes that can fail at once
var maxFailureCount = 0;
// The recovery delay in clock ticks
var moteRecoveryDelay = 5000;
// The failure probability for a single node (1/this value)
var moteFailureProbability = 100;

// The random generator for failing motes (tied to the sim seed)
var random = new Random(sim.getRandomSeed());
// The failure mode for this simulation
var failureMode = "random";

// Simulation file output object
var outputs = new Object();
// Simulation log file prefix
var filePrefix = "log_";

var allMotes = sim.getMote